### Data for today's games

In [91]:
from nba_api.live.nba.endpoints import scoreboard

# Fetch the scoreboard data
games = scoreboard.ScoreBoard()

# Extract raw JSON data
games_data = games.get_dict()

# Parse and display relevant game data
for game in games_data['scoreboard']['games']:
    home_team = game['homeTeam']['teamName']
    home_score = game['homeTeam']['score']
    away_team = game['awayTeam']['teamName']
    away_score = game['awayTeam']['score']
    print(f"{away_team} ({away_score}) @ {home_team} ({home_score}) ")


Wizards (0) @ Hornets (0) 
Nets (0) @ Pistons (0) 
Kings (0) @ Rockets (0) 
Spurs (0) @ Grizzlies (0) 
Warriors (0) @ 76ers (0) 
Bucks (0) @ Mavericks (0) 


### Data for past games

In [92]:
from datetime import date
import nba_api.stats.endpoints as endpoints
from nba_api.live.nba.endpoints import boxscore

# endpoints.scoreboardv2.ScoreboardV2 only works for games from the 2020 season and beyond
game_date = date(2024, 12, 16)
games = endpoints.scoreboardv2.ScoreboardV2(game_date=game_date)
games_data = games.get_data_frames()[0]

for game_id in games_data['GAME_ID']:
    # Get BoxScore
    box = boxscore.BoxScore(game_id) 
    game_data = box.game.get_dict()

    # Extract team scores
    home_team = game_data['homeTeam']['teamName']
    away_team = game_data['awayTeam']['teamName']
    home_score = game_data['homeTeam']['score']
    away_score = game_data['awayTeam']['score']

    # Print the result
    print(f"{away_team} ({away_score}) @ {home_team} ({home_score})")

Heat (124) @ Pistons (125)
76ers (121) @ Hornets (108)
Bulls (122) @ Raptors (121)
Cavaliers (130) @ Nets (101)
Nuggets (130) @ Kings (129)
Jazz (107) @ Clippers (144)


### Boxscore For a Game

In [99]:
from datetime import date
import nba_api.stats.endpoints as endpoints
from nba_api.live.nba.endpoints import boxscore

game_date = date(2024, 12, 16)
games = endpoints.scoreboardv2.ScoreboardV2(game_date=game_date)
games_data = games.get_data_frames()[0]

# Get stats for the first game in the list
game_id = games_data['GAME_ID'][2]

# Get all stats using BoxScoreTraditionalV2
boxscore = endpoints.boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
all_data_frames = boxscore.get_data_frames()

with open('boxscore.txt', 'w') as file:
    for df in all_data_frames:
        file.write(df.to_string())
        file.write('\n\n')

### How to get the season from the game id

In [62]:
def get_season(game_id):
    """
    Determines the NBA season based on the given game_id.

    The game_id follows a pattern where:
    - A game_id starting with '00246' to '00299' corresponds to seasons from 1946-47 to 1999-00.
    - A game_id starting with '00200' to '00224' corresponds to seasons from 2000-01 to 2024-25.

    Logic:
    - Extract the 4th and 5th digits of game_id (game_year).
    - If game_year is between 46 and 99, it belongs to the 1900s (1946-47 to 1999-00).
    - Otherwise, it belongs to the 2000s (2000-01 onward).
    """

    game_year = int(game_id[3:5])  # Extracts the season identifier
    if 46 <= game_year <= 99:
        return f"{game_year + 1900}-{game_year + 1901}"
    else:
        return f"{game_year + 2000}-{game_year + 2001}"

# Test the function
game_id = '0024600001'
print(get_season(game_id))  # Output: 1946-1947
game_id = '0029900001'
print(get_season(game_id))  # Output: 1999-2000
game_id = '0020000001'
print(get_season(game_id))  # Output: 2000-2001
game_id = '0022400001'
print(get_season(game_id))  # Output: 2024-2025

1946-1947
1999-2000
2000-2001
2024-2025


### Determine the type of game based on the game id

#### XXXYYGGGGG
XXX: Game type identifier
- 001 = Preseason
- 002 = Regular Season
- 003 = All-Star Game (Also includes Rising Stars games)
- 004 = Playoffs
- 005 = Play-In Tournament (introduced in 2020)
- 006 = NBA Cup Final (introduced in 2023)

YY: Season year indicator
- 46 -> 99 = 1946-47 -> 1999-00
- 00 -> 24 = 2000-01 -> 2024-2025

GGGGG: Unique game number

### Get the Home and Away Team Given Game ID

In [ ]:
from nba_api.stats.endpoints import boxscoresummaryv2

def get_home_away_team(game_id):
    boxscore = boxscoresummaryv2.BoxScoreSummaryV2(game_id=game_id)
    game_data = boxscore.get_data_frames()[0]  # Game summary data

    home_team_id = game_data['HOME_TEAM_ID'].iloc[0]  # Extract home team ID
    away_team_id = game_data['VISITOR_TEAM_ID'].iloc[0]  # Extract away team ID

    return home_team_id, away_team_id

game_id = "0022401223"
home_team_id, away_team_id = get_home_away_team(game_id)
print("Home Team ID:", home_team_id)
print("Away Team ID:", away_team_id)

Home Team ID: 1610612761
Away Team ID: 1610612741


### Testing

In [120]:
from datetime import date
import pandas as pd
import nba_api.stats.endpoints as endpoints
from nba_api.live.nba.endpoints import boxscore

teams_table = pd.DataFrame(columns=['team_id', 'season_year', 'team_location', 'team_name', 'team_abbreviation'])

game_date = date(2024, 12, 16)
games = endpoints.scoreboardv2.ScoreboardV2(game_date=game_date)
games_data = games.get_data_frames()[0]

# Get stats for the first game in the list
game_id = games_data['GAME_ID'][2]

# Get all stats using BoxScoreTraditionalV2
boxscore = endpoints.boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
player_stats = boxscore.get_data_frames()[0]
team_stats = boxscore.get_data_frames()[1]

# Get general data
game_id = player_stats['GAME_ID'][0]
season_year = get_season(game_id)
home_team_id, away_team_id = get_home_away_team(game_id)

# Get home team data
home_team_row = team_stats[team_stats['TEAM_ID'] == home_team_id].iloc[0]
home_team_location = home_team_row['TEAM_CITY']
home_team_name = home_team_row['TEAM_NAME']
home_team_abbrev = home_team_row['TEAM_ABBREVIATION']

# Get away team data
away_team_row = team_stats[team_stats['TEAM_ID'] == away_team_id].iloc[0]
away_team_location = away_team_row['TEAM_CITY']
away_team_name = away_team_row['TEAM_NAME']
away_team_abbrev = away_team_row['TEAM_ABBREVIATION']

# Convert the data to a data frame and concatenate it with the existing teams_table
new_rows = pd.DataFrame([
    {'team_id': home_team_id, 'season_year': season_year,
     'team_location': home_team_location, 'team_name': home_team_name,
     'team_abbreviation': home_team_abbrev},

    {'team_id': away_team_id, 'season_year': season_year,
     'team_location': away_team_location, 'team_name': away_team_name,
     'team_abbreviation': away_team_abbrev}
])

# Ensure uniqueness before concatenation (set lookup is O(1) time complexity)
existing_keys = set(zip(teams_table['team_id'], teams_table['season_year']))
new_rows_filtered = new_rows[~new_rows.apply(lambda row: (row['team_id'], row['season_year']) in existing_keys, axis=1)]

# Concatenate only if new unique rows exist
if not new_rows_filtered.empty:
    teams_table = pd.concat([teams_table, new_rows_filtered], ignore_index=True)

print(teams_table)

      team_id season_year team_location team_name team_abbreviation
0  1610612761   2024-2025       Toronto   Raptors               TOR
1  1610612741   2024-2025       Chicago     Bulls               CHI


### Team stats for each game in a specified season

In [1]:
from nba_api.stats.endpoints import teamgamelog
import pandas as pd

# Dallas Mavericks team ID
team_id = 1610612742

# Get team game log for the entire season
team_game_log = teamgamelog.TeamGameLog(team_id=team_id, season='2024-25')
team_game_log_data = team_game_log.get_data_frames()[0]

with open('mavericks_season_game_log.txt', 'w') as file:
    file.write(team_game_log_data.to_string())

### Player data

In [5]:
from nba_api.stats.endpoints import playercareerstats

career = playercareerstats.PlayerCareerStats(player_id=2544)

lebron_stats = career.get_data_frames()[0]

print(lebron_stats)

    PLAYER_ID SEASON_ID LEAGUE_ID     TEAM_ID TEAM_ABBREVIATION  PLAYER_AGE  \
0        2544   2003-04        00  1610612739               CLE        19.0   
1        2544   2004-05        00  1610612739               CLE        20.0   
2        2544   2005-06        00  1610612739               CLE        21.0   
3        2544   2006-07        00  1610612739               CLE        22.0   
4        2544   2007-08        00  1610612739               CLE        23.0   
5        2544   2008-09        00  1610612739               CLE        24.0   
6        2544   2009-10        00  1610612739               CLE        25.0   
7        2544   2010-11        00  1610612748               MIA        26.0   
8        2544   2011-12        00  1610612748               MIA        27.0   
9        2544   2012-13        00  1610612748               MIA        28.0   
10       2544   2013-14        00  1610612748               MIA        29.0   
11       2544   2014-15        00  1610612739       